In [ ]:
"""
Connect with Google Drive
"""
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# IF required show the progress bar 

# !pip3 install tqdm 

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import collections
from PIL import Image
from torch.utils.data import Dataset, DataLoader,WeightedRandomSampler,random_split
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm, trange
import numpy as np
import os
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
save_folder = '/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/model'
os.makedirs(save_folder, exist_ok=True)
import cv2
import random
from matplotlib import pyplot as plt
import pandas as pd 
import matplotlib.image as mp
import csv
from random import shuffle 
from collections import defaultdict
from sklearn.model_selection import train_test_split

# DA


> Rotate, Flip, Noise






In [ ]:
"""
Create more data to reduce the imbalance and training data 
"""
# def sp_noise(image,prob):

#     '''
#     prob:
#     '''

#     output = np.zeros(image.shape,np.uint8)

#     thres = 1 - prob

#     for i in range(image.shape[0]):

#         for j in range(image.shape[1]):

#             rdn = random.random()

#             if rdn < prob:

#                 output[i][j] = 0

#             elif rdn > thres:

#                 output[i][j] = 255

#             else:

#                 output[i][j] = image[i][j]

#     return output

# def gasuss_noise(image, mean=0, var=0.001):

#     '''
#         mean : 
#         var : 
#     '''
#     image = np.array(image/255, dtype=float)

#     noise = np.random.normal(mean, var ** 0.5, image.shape)

#     out = image + noise

#     if out.min() < 0:

#         low_clip = -1.

#     else:

#         low_clip = 0.

#     out = np.clip(out, low_clip, 1.0)

#     out = np.uint8(out*255)

#     return out

# x = ['1']
# normal = []
# name = []
# tmp = 6000
# num = 6000
# for j in x:
#   fh = open('/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/Raw_train.csv', 'r')
#   fp = open("/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train.csv", "a")
#   for line in fh:
#     line = line.rstrip()
#     words = line.split(',')
#     normal.append(words[0])
#     name.append(words[1])
#   idx = 0
#   normal = normal[1:]
#   name = name[1:]
#   for i in normal:
#     # if stop >= 144:
#     #   break
#     # stop += 1

#     path = "/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/Raw_train/"+i+".png"
#     raw_image = Image.open(path)

#     rotate_90 = raw_image.rotate(90)
#     rotate_180 = raw_image.rotate(180)
#     rotate_270 = raw_image.rotate(270)

#     #旋转结合翻转
#     flip_vertical_raw = raw_image.transpose(Image.FLIP_TOP_BOTTOM)
#     flip_vertical_90 = rotate_90.transpose(Image.FLIP_TOP_BOTTOM)
#     flip_vertical_180 = rotate_180.transpose(Image.FLIP_TOP_BOTTOM)
#     flip_vertical_270 = rotate_270.transpose(Image.FLIP_TOP_BOTTOM)

#     img=cv2.imread(path)
#     out2 = gasuss_noise(img, mean=0, var=0.001)
#     out1 = sp_noise(img, prob=0.02)

#     #存储
#     flip_vertical_raw.save("/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/train/"+str(num)+".png")
#     flip_vertical_90.save("/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/train/"+str(num+1)+".png")
#     flip_vertical_180.save("/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/train/"+str(num+2)+".png")
#     flip_vertical_270.save("/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/train/"+str(num+3)+".png")
#     mp.imsave('/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/train/'+str(num+4)+'.png',out1)
#     rotate_90.save("/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/train/"+str(num+5)+".png")
#     rotate_180.save("/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/train/"+str(num+6)+".png")
#     rotate_270.save("/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/train/"+str(num+7)+".png")
#     mp.imsave('/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/train/'+str(num+8)+'.png',out2)
#     num+=9

    
#     for k in range(tmp,num):
      
#       # 寫入 This is a testing! 到檔案
#       print(f"{k},{name[idx]}",file=fp)
    
#     # 關閉檔案
#     tmp = num
#     idx += 1
#   fp.close()
#   fh.close()


'\nCreate more data to reduce the imbalance and training data \n'

# Parameter



In [ ]:
# Parameter
NUM_CLASSES = 4

BATCH_SIZE = 128

MAX_EPOCH = 20

RESUME_EPOCH = 0

WEIGHT_DECAY = 5e-4
MOMENTUM = 0.9

LR = 1e-4


BASE = '/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/'


In [ ]:
"""
Load trained model while doing the prediction 
"""
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']  
    model.load_state_dict(checkpoint['model_state_dict'])  
    return model

# Network


In [ ]:
"""
Convultional Neual Network 
Structure:
  Convtional 
  Batch norm
  Relu to delete negative value ) x4 times 
  Linear layer to connect convolutional layer
  output a 4 dimension linear year while classfication is 4.
"""
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(3,16,3,stride=1,padding=1),
        nn.BatchNorm2d(16),
        nn.ReLU(inplace=True),
        nn.Conv2d(16,32,3,stride=1,padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2),

        nn.Conv2d(32,64,3,stride=1,padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.Conv2d(64,128,3,stride=1,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2),

        nn.Conv2d(128,128,3,stride=1,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.Conv2d(128,256,3,stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2)
    )
    self.forwards = nn.Sequential(
        nn.Linear(8*8*256,4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096,1024),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(1024,128),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(128,4),

    )
  def forward(self,x):
    x = self.conv(x)
    # print(x.shape)
    x = x.view(-1,8*8*256)
    x = self.forwards(x)
    return x

model = Net().to('cuda')

model
print("...... Initialize the network done!!! .......")

...... Initialize the network done!!! .......


# Dataset Dataloader


# Create label file of every category 






In [ ]:
"""
Store the label and the path of the images by different categories
And load it into 'data' in numpy arrray type.
"""
SIZE = 1024  # Label size means the data of the imgs of each category 

label = defaultdict(list)
with open('/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train.csv') as f:
  reader = csv.reader(f)
  for row in reader:
    if row[0].isdigit():
      label[row[1]].append(row[0])

# Set up a array
Connective, Immune, Cancer, Normal = [], [], [], []
Connective = np.array(label.get('Connective'))
Immune = np.array(label.get('Immune'))
Cancer = np.array(label.get('Cancer'))
Normal = np.array(label.get('Normal'))
Connective_ =  np.random.choice(Connective, size=SIZE)
Immune_=np.random.choice(Immune, size=SIZE)
Cancer_ = np.random.choice(Cancer, size=SIZE)
Normal_ = np.random.choice(Normal, size=SIZE)

with open('/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/class.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Id','Type'])
    for co in Connective_:
        writer.writerow(['/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/train/'+co+'.png', 0])
    for im in Immune_:
        writer.writerow(['/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/train/'+im+'.png', 1])
    for ca in Cancer_:
        writer.writerow(['/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/train/'+ca+'.png', 2])
    for no in Normal_:
        writer.writerow(['/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/train/train/'+no+'.png', 3])

f.close()
data = pd.read_csv('/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/class.csv')
data = data.to_numpy()
len(data)

4096

# Load label 

# Train data and Validate data


In [ ]:
"""
Separate data into two data: train and validation data 
"""


X, y = data[:, 0], data[:, 1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)# 10% validation 
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# images to tenser and normalize 
transform = transforms.Compose(
    [transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

def default_loader(path):
    img_pil =  Image.open(path)
    # img_pil = img_pil.resize((224,224))
    img_tensor = transform(img_pil.convert('RGB'))
    # img_tensor = img_pil.ToTensor()
    return img_tensor


class trainset(Dataset):
    def __init__(self, file_train, label_train, loader=default_loader):
       
        self.images = file_train
        self.target = label_train
        self.loader = loader

    def __getitem__(self, index):
        fn, label = self.images[index], self
        img = self.loader(fn)
        target = self.target[index]
        return img, target

    def __len__(self): # will return the images length compare with dataset and dataloader 
        return len(self.images)


In [ ]:
"""
Train data parameter 

Batch size
epoch
iterate number 
loss_plot: store the loss of train data every epoch 
accuracy_plot: store the accory of train data every epoch 
valid_loss_plot: store the loss of valid data every epoch 
valid_accuracy_plot: store the valid accuracy of valid data every epoch
 
"""

batch_size = BATCH_SIZE

max_batch = len(X_train)//batch_size    # how many 
epoch_size = len(X_train) // batch_size #  how many batch of one epoch
print(epoch_size) 

max_iter = MAX_EPOCH * epoch_size
start_iter = RESUME_EPOCH * epoch_size
epoch = RESUME_EPOCH


# Plot loss, accuracy
loss_plot = []
accuracy_plot = []
valid_loss_plot = []
valid_accuracy_plot = []

# learning rate 
lr = LR

# optimizer will adjust the learning rate or other hyperparameter 
optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=WEIGHT_DECAY)
# optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM,weight_decay=WEIGHT_DECAY)

# Loss function 
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss().cuda()
# criterion = LabelSmoothSoftmaxCE()
# criterion = LabelSmoothingCrossEntropy()



# load data into dataloader for training or testing

train_data  = trainset(X_train, y_train)
trainloader = DataLoader(train_data, batch_size,shuffle=True)

val_data  = trainset(X_test, y_test)
valloader = DataLoader(val_data, batch_size,shuffle=True)

28


In [ ]:
## adjust learning rate: become smaller while epoch get higher 
def adjust_learning_rate(learning_rate, learning_rate_decay, optimizer, epoch):
    """Sets the learning rate to the initial LR multiplied by learning_rate_decay(set 0.98, usually) every epoch"""
    learning_rate = learning_rate * (learning_rate_decay ** epoch)

    # update learning rate in optimizer 
    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate

    return learning_rate

# Train

In [ ]:
"""
check trained model is overfitting or underfitting 
"""
def validation():
  model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
  predicts = []
  with torch.no_grad():
    correct = 0
    total = 0
    loss = []
    acc = []
    for images, labels in valloader:
        images = images.cuda()
        labels = labels.cuda()
        outputs = model(images)
        val_loss = criterion(outputs, labels)
        prediction = torch.max(outputs.data, 1)[1]
        val_correct = (prediction == labels).sum()

        val_acc = (val_correct.float()) / len(labels) * 100
        loss.append(val_loss.cpu().numpy())
        acc.append(val_acc.cpu().numpy())
        predicts.append([prediction.tolist()[0], labels.tolist()[0]])
    print(f"Valid Loss: {np.mean(loss)},Valid Acc: {np.mean(acc)}")
  return [np.mean(loss),np.mean(acc)]


In [ ]:
"""
train model 
"""
total_step = len(trainloader)  # total batch trained 
curr_lr = LR                   # local learning rate 
min_valid_loss = np.Inf        # minumn validate loss
for epoch in range(1,MAX_EPOCH+1):
  for i, (images, labels) in enumerate(trainloader):
    model.train()
    images = images.cuda()
    labels = labels.cuda()
    
    
    outputs = model(images)
    loss = criterion(outputs, labels)
    

    # update optimizer and backward loos(BP)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    
    prediction = torch.max(outputs, 1)[1]
    train_correct = (prediction == labels).sum()
# #train_correct is longtensor transform to float
#     print(train_correct.type())
    train_acc = (train_correct.float()) / batch_size * 100

    # validation function to print out validate loss and accuracy 
    val_loss, val_acc = validation()     
    print("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} || Acc: {:.4f}%, LR: {:.8f}"
    .format(epoch, MAX_EPOCH, i+1, total_step, loss.item(),train_acc,curr_lr))
    
    #store the data for plot 
    accuracy_plot.append(train_acc)
    loss_plot.append(loss.item())
    valid_loss_plot.append(val_loss)
    valid_accuracy_plot.append(val_acc)

  # Store model each 5 epoch 
  if epoch % 5 == 0 and epoch > 0:
            checkpoint = {'model': model,
                        'model_state_dict': model.state_dict(),
                        # 'optimizer_state_dict': optimizer.state_dict(),
                        'epoch': epoch}
            torch.save(checkpoint, os.path.join(save_folder, 'epoch_{}.pth'.format(epoch)))
  # Adjust local learning rate 
  curr_lr = adjust_learning_rate(curr_lr, 0.98, optimizer, epoch)

OSError: ignored

# Predict


In [ ]:
"""
Prediction: do the prediction by the trained model 
"""


transform = transforms.Compose(
    [transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
def predict(model):
  model = load_checkpoint(model)
  print('..... Finished loading model! ......')
  lab = {3:'Normal',2:'Cancer',1:'Immune',0:'Connective'}
  if torch.cuda.is_available():
    model.cuda()
    pred_list, _id = [], []
  print('.....Start predicting image.....')
  for i in tqdm(range(10001,10401)):
           
    _id = np.arange(10001,10401)
    path = '/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/test/test/'+str(i)+'.png'
    img = Image.open(path).convert('RGB') #from PIL import Image # load images by path 
    img = transform(img).unsqueeze(0) #transform
    

    if torch.cuda.is_available():
        img = img.cuda()
    # prediction 
    with torch.no_grad():
      out = model(img)
      prediction = torch.argmax(out, dim=1).cpu().item()
      pred_list.append(prediction)
  pred_list = np.array(pred_list)
  print(pred_list)
  pred_list_name = []
  for i in pred_list:
    pred_list_name.append(lab[i])

  return _id, pred_list_name

NameError: ignored

In [ ]:
"""
Store the prediction in csv 

"""
import pandas as pd

trained_model = '/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/model/epoch_20.pth'
# print(sum(accuracy_plot)/len(accuracy_plot))
_id, pred_list = predict(trained_model)
submission = pd.DataFrame({"Id": _id, "Type": pred_list})
print()
submission.to_csv('/content/drive/MyDrive/deep-learning-for-msc-coursework-2021/'+ '{}_submission.csv'
                      .format('LinLeroy'), index=False, header=True)


In [ ]:
"""
Plot the loss, accuracy and the validation part 
"""
import matplotlib.pyplot as plt
from pylab import *  

figure(figsize=(14, 6), dpi=80)
x = range(1,total_step*MAX_EPOCH+1)

plt.plot(x[::2], accuracy_plot[::2], marker='o', mec='r', mfc='w',label='Accuracy')
plt.plot(x[::2], valid_accuracy_plot[::2], marker='.', mec='b', mfc='k',label='Valid Accuracy')
plt.legend(loc = 1)
plt.xlabel("Number")
plt.ylabel("Accuracy")



# Plot


In [ ]:
"""
Plot the loss, accuracy and the validation part 
"""
import matplotlib.pyplot as plt
from pylab import *  

figure(figsize=(14, 6), dpi=80)
x = range(1,total_step*MAX_EPOCH+1)

plt.plot(x[::2], loss_plot[::2], marker='o', mec='r', mfc='w',label='Loss')
plt.plot(x[::2], valid_loss_plot[::2], marker='.', mec='b', mfc='k',label='Valid Loss')
plt.legend(loc = 1)
plt.xlabel("Number")
plt.ylabel("Loss")

In [ ]:
"""
Confusion matrix 
"""
confusion_pred = []
from sklearn.metrics import confusion_matrix
import seaborn as sn

nb_classes = 4

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

# load model 
model = load_checkpoint(trained_model)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(trainloader):
        inputs = inputs.cuda()
        classes = classes.cuda()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())


df_cm = pd.DataFrame(conf_mat, range(4), range(4))
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size


sn.heatmap(df_cm, annot=True,cmap="YlGnBu", annot_kws={"size": 16},
           xticklabels=['Connective','Immune','Cancer','Normal'],
           yticklabels=['Connective','Immune','Cancer','Normal']
           ) # font size

plt.show()

In [ ]:
"""
Confusion matrix 
"""
confusion_pred = []
from sklearn.metrics import confusion_matrix
import seaborn as sn

nb_classes = 4

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

# load model 
model = load_checkpoint(trained_model)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(valloader):
        inputs = inputs.cuda()
        classes = classes.cuda()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())


df_cm = pd.DataFrame(conf_mat, range(4), range(4))
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.color_palette("flare", as_cmap=True)


sn.heatmap(df_cm, annot=True, annot_kws={"size": 16},
           xticklabels=['Connective','Immune','Cancer','Normal'],
           yticklabels=['Connective','Immune','Cancer','Normal']
           ) # font size

plt.show()

# Reference


https://stackoverflow.com/questions/35572000/how-can-i-plot-a-confusion-matrix

